In [1]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.optimize import brentq
import matplotlib.pyplot as plt

# Functions

In [8]:
#parameters
t = 0.425
e=1.602*10**(-19)
kB = 8.617e-5
T=0.1
beta=1/(kB*T)

A0=0.1
Theta=0


# k-grid
n_k = 150
kx = np.linspace(-np.pi, np.pi, n_k)
ky = np.linspace(-np.pi, np.pi, n_k)
KX, KY = np.meshgrid(kx, ky)

In [18]:
#dispersion

def epsilon(t,kx,ky,mu):
    return -2*t*(np.cos(kx)+np.cos(ky))-mu

def Ek(t,kx,ky,mu,delta):
    return np.sqrt(epsilon(t,kx,ky,mu)**2+delta**2*formfactor(kx,ky)**2)

def formfactor(kx,ky):
    return 1/2*(np.cos(kx)-np.cos(ky))

def n(t,mu,delta):
    return 1/n_k *np.sum(1-epsilon(t,KX,KY,mu)/Ek(t,KX,KY,mu,delta)*np.tanh(beta/2*Ek(t,KX,KY,mu,delta)))

def density_difference(mu,n0,t,delta):
    return n(t,mu,delta)-n0

In [12]:
def sigmax_0(kx,ky,mu,delta):
    return delta*formfactor(kx,ky)/(2*Ek(t,kx,ky,mu,delta))*np.tanh(beta/2*Ek(t,KX,KY,mu,delta))

def sigmaz_0(kx,ky,mu,delta):
    return -epsilon(t,kx,ky)/(2*Ek(t,kx,ky,mu,delta))*np.tanh(beta/2*Ek(t,KX,KY,mu,delta))

def delta_0(mu,V,delta):
    return V/n_k*np.sum(formfactor(KX,KY)*sigmax_0(KX,KY,mu,delta))

In [13]:
def deltaDelta(V,x,y):
    return V/n_k*np.sum(formfactor(KX,KY)*(x-y))

def Dk(kx,ky,Theta):
    return np.cos(Theta)**2*2*t*np.cos(kx)+np.sin(Theta)**2*2*t*np.cos(kx)

def emodA(kx,ky,Theta,A_0):
    return e**2*A_0**2*Dk(kx,ky,Theta)

In [27]:
n0=1
delta0=0
V=2

try:
    mu_0 = brentq(density_difference, -10, 10, args=(n0,t,delta0), xtol=1e-5)
except ValueError:
    print("mu not found")

print("mu_0=",mu_0)
mu_current = mu_0

delta_current=0.5



for i in range(1000):
    # Compute updated order parameters
    delta_new=delta_0(mu_current,V,delta_current)

    # Update mu to maintain total density ~1
    try:
        mu_new = brentq(density_difference, -100, 100, args=(n0,t,delta_new), xtol=1e-5)
    except ValueError:
        print("mu not found")
        break

    # Check total density
    n_new=n(t,mu_new,delta_new)

    print(f"step {i+1}: delta = {delta_new:.3f}, μ = {mu_new:.3f}, n = {n_new:.3f}")

    # Check convergence
    if (np.abs(delta_new-delta_current)+np.abs(n_new - n0)) < 1e-5:
        print(f"✅ Converged in {i+1} steps.")
        break

    # Update for next step
    delta_current=delta_new
    mu_current = mu_new

print("\n📌 Final self-consistent results:")
print(f"delta = {delta_current:.6f}")
print(f"μ   = {mu_current:.6f}")
print(f"n   = {n(t,mu_current,delta_current):.6f}")

mu_0= -1.6830261668269308
step 1: delta = 11.904, μ = -51.134, n = 1.000
step 2: delta = 8.656, μ = -37.184, n = 1.000
step 3: delta = 8.656, μ = -37.183, n = 1.000
step 4: delta = 8.656, μ = -37.183, n = 1.000
✅ Converged in 4 steps.

📌 Final self-consistent results:
delta = 8.655621
μ   = -37.183144
n   = 1.000000
